# EM Algorithm - Univariate Gaussian Mixture Model 

In [7]:
import numpy as np
from tqdm import tqdm 
import matplotlib.pyplot as plt

## Estimating 3 Component GMM

In [8]:
N = 1500
D = 2
K = 3
mu_1, sigma_1 = ( np.array([-4, -5]), np.array([[1, 0], [0, 1]]) )
mu_2, sigma_2 = ( np.array([0, 0]), np.array([[1, 0], [0, 1]]) )
mu_3, sigma_3 = ( np.array([4, 5]), np.array([[1, 0], [0, 1]]) )

priors = [0.2, 0.5, 0.3]

X1 = np.random.multivariate_normal(mean=mu_1, cov=sigma_1, size=int(priors[0]*N))
X2 = np.random.multivariate_normal(mean=mu_2, cov=sigma_2, size=int(priors[1]*N))
X3 = np.random.multivariate_normal(mean=mu_3, cov=sigma_3, size=int(priors[2]*N))
X = np.concatenate((X1, X2, X3))
X1.shape, X2.shape, X3.shape, X.shape

((300, 2), (750, 2), (450, 2), (1500, 2))

Probability Density Function of Multivariate Gaussian

In [9]:
def pdf(x, mu, covariance):
    """pdf of the multivariate normal distribution."""
    x_m = x - mu
    d = x.shape[-1]
    return (1. / (np.sqrt((2 * np.pi)**d * np.linalg.det(covariance))) * 
            np.exp(-(np.linalg.solve(covariance, x_m).T.dot(x_m)) / 2))

Random Initialization

In [16]:
lambdas = np.ones((K))/K
means = np.array([np.random.choice(X[:, i], K) for i in range(D)]).transpose(-1, 0) #(K, D)
covariances = np.array([np.identity(D)/K for _ in range(K)])
print('Means: ', means, '\n\n', 'Cov: ', covariances, '\n\n', 'Lambdas: ', lambdas)

Means:  [[-2.15739904  4.73898699]
 [ 4.74832778 -0.44523709]
 [-3.91799656  3.52849593]] 

 Cov:  [[[0.33333333 0.        ]
  [0.         0.33333333]]

 [[0.33333333 0.        ]
  [0.         0.33333333]]

 [[0.33333333 0.        ]
  [0.         0.33333333]]] 

 Lambdas:  [0.33333333 0.33333333 0.33333333]


Iterative EM algorithm (till convergence)

In [17]:
for iteration in range(100):
    G = np.zeros((K, N)) #Gamma

    #E-Step
    for k in range(K):
        G[k, :] = np.array([pdf(X_i, means[k], covariances[k]) * lambdas[k] for X_i in X])
    G = G / np.sum(G, axis=0)
  
    #M-Step
    means = np.matmul(G, X)
    for k in range(means.shape[0]):
        means[k] /= np.sum(G, axis=1)[k]
    means += 1e-18
    print(means.shape)

    for k in range(K):
        numerator = 0
        for i in range(N):
            Y = (X[i] - means[k]).reshape(D, 1)
            numerator += G[k, i] * np.matmul(Y, Y.T)
        covariances[k] = numerator / np.sum(G[k]) 
        lambdas[k] = np.mean(G[k])
    
    print(f'Iteration {iteration}: \nmeans - {means}, \nvariances - {covariances}, \nlambdas = {lambdas}\n')

print(f'Results\n: means - {means}, \nvariances - {covariances}, \nlambdas = {lambdas}\n')

(3, 2)
Iteration 0: 
means - [[ 1.42590476  3.42655225]
 [ 2.05203288  1.52641669]
 [-3.00502352 -2.83714139]], 
variances - [[[2.9795833  3.65103674]
  [3.65103674 5.18109952]]

 [[4.64163323 5.88083343]
  [5.88083343 8.65056922]]

 [[2.70250554 3.61429147]
  [3.61429147 6.55609347]]], 
lambdas = [0.17108922 0.51952447 0.30938631]

(3, 2)
Iteration 1: 
means - [[ 1.24122723  3.06283248]
 [ 1.97109293  1.52588989]
 [-2.8802498  -2.69380307]], 
variances - [[[3.62238364 4.42637704]
  [4.42637704 6.23245745]]

 [[4.99810156 6.17900095]
  [6.17900095 9.02471038]]

 [[3.28935204 4.30208794]
  [4.30208794 7.59740372]]], 
lambdas = [0.16541398 0.53156932 0.3030167 ]

(3, 2)
Iteration 2: 
means - [[ 1.1503019   2.87892587]
 [ 1.94033901  1.54295312]
 [-2.8228256  -2.64786696]], 
variances - [[[3.91829841 4.75570177]
  [4.75570177 6.67920771]]

 [[5.13748796 6.26252031]
  [6.26252031 9.11982814]]

 [[3.53328808 4.56580689]
  [4.56580689 7.99351291]]], 
lambdas = [0.16301335 0.53650841 0.300478